In [1]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import os
import torch
import random
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import Image

SEED = 24535


def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    # torch.backends.cudnn.deterministic = True


seed_everything()


In [2]:
pd.set_option('display.max_rows', 500)


In [3]:
train = pd.read_csv('data_set_ALL_AML_train.csv')


In [4]:
test = pd.read_csv('data_set_ALL_AML_independent.csv')


In [5]:
train.shape, test.shape


((7129, 78), (7129, 70))

In [6]:
def get_person_columns(df):
    return [c for c in df.columns if c.isdigit()]


In [7]:
target_orig = pd.read_csv('actual.csv')
target_orig.shape


(72, 2)

In [8]:
train_expressions_cols = get_person_columns(
    train)  # list(map(str,range(1,38)))
test_expressions_cols = get_person_columns(
    test)  # list(map(str,range(39, 63)))
# expressions_cols


In [9]:
# train_expressions_cols


In [10]:
# train['Gene Accession Number']


In [11]:
train_exp = pd.DataFrame(train[train_expressions_cols]).set_index(
    train['Gene Accession Number'])
test_exp = pd.DataFrame(test[test_expressions_cols]).set_index(
    test['Gene Accession Number'])


In [12]:
import plotly.graph_objects as go

# fig = go.Figure(data=[go.Histogram(x=exp.values.reshape(-1))])
# fig.show()


In [13]:
X_train = train_exp.rename(
    {c: f"person{c}" for c in train_exp.columns}, axis='columns').T.sample(frac=1)
X_test = test_exp.rename(
    {c: f"person{c}" for c in test_exp.columns}, axis='columns').T
val_size = 16
X_val = X_train[-val_size:]
X_train = X_train[:-val_size]


In [14]:
X_train.shape, X_val.shape, X_test.shape


((22, 7129), (16, 7129), (34, 7129))

In [15]:
target = pd.Series(target_orig.cancer.values, index=[
                   f"person{n}" for n in target_orig.patient]).rename("cancer")
target_one_hot = (target == "ALL").astype(int)
target_one_hot.value_counts()
Y_train = target_one_hot.loc[X_train.index]
Y_val = target_one_hot.loc[X_val.index]
Y_test = target_one_hot.loc[X_test.index]


In [16]:
import phik
from phik import resources, report


In [27]:
from tqdm import tqdm
phik_cors = {}
for c in tqdm(X_train.columns):
    col_cor = pd.concat([X_train[c], Y_train], axis=1).phik_matrix(verbose=False)[
        'cancer'].iloc[:-1]
    phik_cors[c] = col_cor[0]
phik_cors = pd.Series(phik_cors)


# [['M84526_at']]


100%|██████████| 7129/7129 [02:46<00:00, 42.82it/s]


In [29]:
phik_cors.sort_values()[-10:]


U78190_rna1_at      0.987377
U82275_at           0.991114
U41813_at           0.993868
Y00787_s_at         1.000000
X95735_at           1.000000
U80457_at           1.000000
L27584_s_at         1.000000
U32315_at           1.000000
U22376_cds2_s_at    1.000000
HG2788-HT2896_at    1.000000
dtype: float64

In [30]:
phik_imp_features = phik_cors[phik_cors > 0.75].index
phik_imp_features


Index(['AFFX-CreX-5_st', 'AFFX-DapX-5_at', 'AFFX-HUMGAPDH/M33197_3_at',
       'AFFX-HUMTFRR/M11507_5_at', 'AFFX-HUMTFRR/M11507_M_at',
       'AFFX-HUMTFRR/M11507_3_at', 'A28102_at', 'AB000467_at', 'AB002382_at',
       'AB006190_at',
       ...
       'L32961_at', 'U20499_at', 'U62434_at', 'X06318_at', 'X51345_at',
       'D38437_f_at', 'J00212_f_at', 'M37755_f_at', 'L10717_at', 'U29175_at'],
      dtype='object', length=454)

In [41]:
# pd.Series(phik_imp_features).to_csv("phik_important.csv")

In [ ]:
cors = X_train.apply(lambda x: x.corr(target_one_hot))


In [ ]:
# persons.join(target)


In [31]:
# cors = cors.sort_values()
# cors


In [ ]:
important_features = cors[cors.abs() > 0.7].index


In [ ]:
# cors.iloc[:30]


In [ ]:
# cors.iloc[-15:]


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [42]:
X_train = X_train[phik_imp_features]
X_val = X_val[phik_imp_features]
X_test = X_test[phik_imp_features]


In [44]:
reg = LogisticRegression()
reg.fit(X_train, Y_train)
predicted = reg.predict(X_val)


In [45]:
predicted


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [37]:
from sklearn.metrics import classification_report


In [46]:
print(classification_report(Y_val, predicted))


              precision    recall  f1-score   support

           0       0.92      0.86      0.89        14
           1       0.90      0.95      0.93        20

    accuracy                           0.91        34
   macro avg       0.91      0.90      0.91        34
weighted avg       0.91      0.91      0.91        34

